In [1]:
import sys, os, math
import numpy as np
sys.path.insert(0,os.path.expanduser('~/git/pyzx')) # git version
sys.path.insert(0,'/workspaces/pyzx')
import pyzx as zx
from pyzx import print_matrix
from pyzx.basicrules import *
import time
from fractions import Fraction
import random
from pyzx.gflow import gflow
from pyzx.utils import vertex_is_zx
from pyzx.graph.base import BaseGraph
from pyzx import VertexType, EdgeType
# zx.settings.drawing_backend = 'matplotlib'

Z = zx.VertexType.Z
X = zx.VertexType.X
B = zx.VertexType.BOUNDARY
SE = zx.EdgeType.SIMPLE
HE = zx.EdgeType.HADAMARD

In [2]:
def transpose(p):
    pt = dict()
    for k,s in p.items():
        for v in s:
            if v not in pt: pt[v] = set()
            pt[v].add(k)
    return pt
def extend(g, s):
    s1 = s.copy()
    for v in s:
        s1 |= {v for v in g.neighbors(v) if vertex_is_zx(g.type(v))}
    return s1
def extend_dict(g, p):
    pe = dict()
    for k,s in p.items():
        pe[k] = extend(g, s)
    return pe
def is_dag(p):
    pt = transpose(p)
    for k,s in p.items():
        for v in s:
            if k != v and k in pt and v in pt[k]:
                return False
    return True

def is_clifford(p):
    return p in (0, 1, Fraction(1, 2), Fraction(3,2))

def gadgetize(g: BaseGraph):
    for v in list(g.vertices()):
        p = g.phase(v)
        if not is_clifford(p) and g.vertex_degree(v) > 1:
            x = g.add_vertex(VertexType.Z, -1, g.row(v))
            y = g.add_vertex(VertexType.Z, -2, g.row(v))
            g.add_edge((x, y), EdgeType.HADAMARD)
            g.add_edge((v, x), EdgeType.HADAMARD)
            g.set_phase(y, p)
            g.set_phase(v, 0)


In [3]:
random.seed(1330)
g = zx.generate.cliffordT(3, 60, p_t=0.2)
zx.draw(g)
g

Graph(94 vertices, 113 edges)

In [4]:
g1 = g.copy()
zx.full_reduce(g1)
gadgetize(g1)
g1.normalize()
g1 = g1.copy()
zx.to_rg(g1)
# g2 = g1.copy()
# for e in list(g2.edges()): zx.hrules.had_edge_to_hbox(g2, e)
zx.draw(g1, labels=True)

In [7]:
o, p = gflow(g1, focus=True, reverse=True, pauli=True)
vs = [v for v in g1.vertices() if not is_clifford(g1.phase(v))]
list.sort(vs, key=lambda v: o[v], reverse=True)
for v in vs: print(f"{v} <- {p[v]} + {extend(g1, p[v]).difference(p[v])}")

17 <- {16} + {17, 3}
19 <- {18} + {19, 4}
15 <- {3, 4, 6, 7, 17, 19} + {5, 8, 9, 10, 15, 16, 18}
21 <- {19, 4, 20, 6} + {3, 5, 9, 18, 21}
25 <- {24, 19, 4} + {9, 18, 25}
23 <- {4, 5, 19, 21, 22} + {3, 6, 8, 9, 18, 20, 23}
27 <- {3, 5, 6, 17, 21, 26} + {8, 9, 10, 16, 20, 27}


In [6]:
o,p = gflow(g1, focus=True, reverse=False, pauli=True)
pt = transpose(p)
print(o)
pt

{9: 4, 10: 4, 11: 4, 8: 3, 15: 3, 23: 3, 25: 3, 27: 3, 3: 2, 5: 2, 7: 2, 17: 2, 21: 2, 22: 2, 24: 2, 26: 2, 4: 1, 6: 1, 16: 1, 19: 1, 20: 1, 18: 0}


{11: {3, 4, 5, 8, 15, 17, 19, 21, 23},
 7: {15},
 22: {23},
 24: {25},
 26: {27},
 27: {3, 5, 6, 17, 21, 26},
 10: {3, 5, 6, 17, 21},
 15: {3, 4, 6, 7, 17, 19},
 8: {4, 5, 19, 21},
 23: {4, 5, 19, 21, 22},
 16: {17},
 20: {21},
 25: {4, 19, 24},
 5: {4, 6, 19},
 9: {4, 19},
 21: {4, 6, 19, 20},
 17: {16},
 18: {19},
 19: {18}}